In [5]:
from QuantLib import *

In [6]:
option = EuropeanOption(PlainVanillaPayoff(Option.Call, 100.0), EuropeanExercise(Date(7, March, 2019)))

In [7]:
print(option.NPV)

<bound method Instrument.NPV of <QuantLib.QuantLib.EuropeanOption; proxy of <Swig Object of type 'boost::shared_ptr< EuropeanOption > *' at 0x00000201D7E963C0> >>
